## In this notebook you can ask the RAG model the question you want, keep in mind the documents that we used were financial news from the first quarter of 2018

First set the working directory to scripts

In [1]:
import os
os.chdir("scripts")

# (Optional) Embed the data

Here you can embed the data, this process may take a long time and we strongly advise to download the already embedded data from [here](https://epflch-my.sharepoint.com/:f:/g/personal/marco_giuliano_epfl_ch/EomT1E_2ZaFCkB7zpx-jDAMBxLz6UP8NGtdBNvK10RaYHQ?e=w8s5QY).\
If you want to embed the data yourself we advise you to use a GPU, otherwis it can take some days!

In [11]:
from embed import embed
from datasets import load_dataset

In [ ]:
ds = load_dataset("ashraq/financial-news-articles")
print("Data downloaded!")

Embed wtih BGE

In [ ]:
model = "BGE"
path = "../embeddings/save"

embed(model, path, ds)

Embed with BGE_Finetuned

In [ ]:
model = "BGE_Finetuned"
path = "../embeddings/save_finetuned"

embed(model, path, ds)

# Do the RAG

Once you have the embedded data in the "embeddings" folder you can start retrieving the data and generating responses based on a query.\
If you want to try to use the BGE pretrained model make sure you have the embeddings saved in embeddings/save.\
If you want to try to use the Finetuned BGE  model make sure you have the embeddings saved in embeddings/save_finetuned and the model weights saved in models/BGE_Finetuned.

In [2]:
import chromadb
from llama_index.core import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from utils import choose_embed_model
from llama_index.core import VectorStoreIndex
from llama_index.core.schema import TextNode
from models import retrieve, answer
from utils import data_to_df

resource module not available on Windows


To use reranking you need to set a cohere api key, not necessay otherwise (But still run the cell)

In [3]:
cohere_api_key = "COHERE_API_KEY"

The openai key is necessary to generate the response from the LLM, please put yours here

In [4]:
import openai
openai_api_key = "sk-"
openai.api_key = openai_api_key

Choose the model here between: \
BM25 set "BM25" \
BGE set "BGE" \
Finetuned BGE set "BGE_Finetuned" \
Hybrid search with BGE and BM25 set "Hybrid_BGE" \
Hybrid search with Finetuned BGE and BM25 set "Hybrid_Finetuned"

In [5]:
model = "Hybrid_Finetuned"

In [6]:
top_k = 10                  # Number of documents to retrieve
alpha = 0.9                 # Alpha value for the Hybrid retrieval, 0.9 is the optimal value we found
query_rewriting = False     # Whether to use query rewriting or not
reranking = False           # Whether to use reranking or not
top_k_before_reranking = 50 # Number of documents to retrieve before reranking

In [7]:
path = False

if model == "BGE_Finetuned" or model == "Hybrid_Finetuned":
    path = "../embeddings/save_finetuned"
elif model == "BGE" or model == "Hybrid_BGE":
    path = "../embeddings/save"

index = None

if path:
    chroma_client = chromadb.PersistentClient(path=path)
    chroma_collection = chroma_client.get_or_create_collection("mydocs")
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)


    embed_model = choose_embed_model(model)
    index = VectorStoreIndex.from_vector_store(
        vector_store, 
        storage_context=storage_context,
        embed_model = embed_model,
        show_progress=True,
    )
    print("Embedding loaded!")

You try to use a model that was created with version 3.3.1, however, your version is 3.3.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





Embedding loaded!


To use the BM25 model we need the not embedded data, please download this data with this cell:

In [8]:
raw_nodes = None
if model == "BM25" or model == "Hybrid_Finetuned" or model == "Hybrid_BGE":
    from datasets import load_dataset
    ds = load_dataset("ashraq/financial-news-articles")
    raw_nodes = [TextNode(text=f"Title: {row['title']}\nContent: {row['text']}") for row in ds['train']]

# Now we can ask a question to the model

You can now modify the query to ask any question

In [13]:
query = "What does the Federal Reserve say on interest rates?"

In [14]:
retrieved_nodes = retrieve(model, 
                           query, 
                           index, 
                           raw_nodes, 
                           top_k, 
                           alpha=alpha, 
                           query_rewriting=query_rewriting, 
                           openai_api_key=openai_api_key, 
                           reranking=reranking, 
                           top_k_before_reranking=top_k_before_reranking, 
                           cohere_api_key=cohere_api_key)

In [15]:
response = answer(query, retrieved_nodes)

In [16]:
print(response)

The Federal Reserve decided to keep interest rates unchanged at its recent meeting, but expressed confidence that a recent rise in inflation to near its 2% target would be sustained. This leaves the Federal Reserve on track to raise borrowing costs in the near future, with expectations of further gradual increases in the federal funds rate.


We can also see which nodes were used by the LLM to answer the question

In [ ]:
data_to_df([n.node for n in retrieved_nodes])